In [21]:
# Import libraries                                                                                                                                                                                                                                                                                                                                                                                                                        import pandas as pd 
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
# url = "https://www.zomato.com/mumbai/great-food-no-bull"
# path = r'C:\Users\Deepthi Poojary\Web Scrapping\chromedriver'
# drive = webdriver.Chrome(path)
# drive.implicitly_wait(30)
# drive.get(url)
# source = drive.page_source

In [3]:
# py_button = drive.find_elements_by_css_selector("div[class = 'col-s-8 col-l-1by3']") #.get_attribute("href")
# print(len(py_button))

In [22]:
# Open tab in full screen (maximized)
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")

In [33]:
# Instantiate Selenium Webdriver
driver = webdriver.Chrome(options = options , executable_path = r'C:\Users\Deepthi Poojary\Web Scrapping\chromedriver')

# Get the main page (Top Restaurants in Mumbai on Zomato)
driver.get("https://www.zomato.com/mumbai/great-food-no-bull")

# Get page soure
source = driver.page_source

In [34]:
# Get the main page/window handle after it has opened
main_window = driver.window_handles[0]

In [35]:
# Get title of  main window
main_window_title = driver.title
print(main_window_title)

Best Food in Mumbai | Zomato


In [36]:
# Calculating number of restaurant on the main page using BeautifulSoup and main page source
soup = BeautifulSoup(source,"html.parser") # col-s-8 col-l-1by3
type(soup)
button = soup.find_all("div", attrs = {"class":"col-s-8 col-l-1by3"})
bs_len = len(button)
bs_len

183

This is the number of restaurant currently on the main page.

In [37]:
# Calculating number of restaurants on the main page using Selenium's find_elements_by_css_selector 
links = driver.find_elements_by_css_selector("div[class = 'col-s-8 col-l-1by3']") #driver.find_element_by_css_selector("div[data_res_position = '1']")  #WebDriverWait(driver,20).until(EC.element_to_be_clickable(("div[data-res-position = '1']")))
sel_len = len(links)
sel_len

183

This is just to confirm the number of restaurants on main page and hence bs_len should be equal to sel_len.

In [28]:
# # selecting a particular restaurant from main page
# py_button = drive.find_elements_by_css_selector("div[data-res-position = '1']") #.get_attribute("href")
# print(len(py_button))
# #print(py_button)
# #print(py_button.text)
# #print(py_button.get_attribute("a"))

In [38]:
#  Data we'll be extracting
# Made a dictionary for ease of structuring dataframe
table = {"Name":[],"Area":[],"Type":[],"Rating":[],"Ph_Number":[],"Cuisine":[],"Avg_price":[],"Beer":[],"Time":[],"Adress":[],"Facility":[],"Food_Rating":[],"Service_Rating":[],"Ambience_Rating":[] }

In [ ]:
for link in links:        # for loop loops through links of all restaurants in links variable

    
    ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()        # Opens each restaurants link in new tab by  "ctrl" + clicking (link) 
    
    # get the window handle after a new window/tab has opened
    new_window = driver.window_handles[1]
    
    # Switch focus to new window so that required operations can be done in that tab
    driver.switch_to.window(new_window)
    
    # get new window title
    new_window_title = driver.title
    
    if main_window_title != new_window_title:     # Check if main_window and new_window are different
        
        # Providing page source of new window
        Source = driver.page_source

        # BeautifulSoup 
        soup1 = BeautifulSoup(Source, "html.parser")

        ### Gate 1
        header = soup1.find("div", attrs =  {"class" : "col-l-12"})
        # Restaurant Name
        Name = soup1.find("div", attrs = {"class" : "col-l-12"}).h1.text.strip('\n').strip(" ")
        table["Name"].append(Name)
        # Main adress
        Area = header.div.a.text.strip(" ").strip("\n").strip(" ")
        table["Area"].append(Area)
        # Type of Restaurant
        Type = header.find("span", attrs = {"class" : "res-info-estabs grey-text fontsize3"}).text.strip("\n")
        table["Type"].append(Type)

        
        ### Gate 2
        header2 = soup1.find("div"  , attrs = {"class" : "col-l-4 tac left"})
        #Rating of restaurent
        Rating = header2.find("div" , attrs = {"tabindex" : "0"}).text.strip().replace(" ","").replace("\n","")
        table["Rating"].append(Rating)

        
        ### Gate 3
        tail1 = soup1.find("div" , class_ = "col-l-1by3 pl0 pr20")
        #Phone Number 
        Ph_Number = tail1.find("span" , attrs = {"class" : "fontsize2 bold zgreen"}).text
        table["Ph_Number"].append(Ph_Number)
        #Cusines
        Cuisine = tail1.find("div" , class_ = "res-info-cuisines clearfix").text
        table["Cuisine"].append(Cuisine)
        #Average  price or price for two person
        Avg_price = tail1.find("div" , class_ = "res-info-detail").find("span" ,attrs = {"tabindex" : "0"}).text.split("for")[0]
        table["Avg_price"].append(Avg_price)
        #Beer price/pint
        Beer = tail1.find("div" , class_ = "res-info-detail").find("div",class_ = "mt5").text.strip("\n").split(" ")[0]
        table["Beer"].append(Beer)

        
        ### Gate 4
        tail2= soup1.find("div",attrs={"class" : "col-l-1by3 pl20 pr20"})
        # Today's timing
        Time = tail2.find("div",attrs= {"class":"clearfix"}).find("div",class_ = "medium").text.strip("Today")
        table["Time"].append(Time)
        # Full Adress
        Address = tail2.find("div" , class_ = "resinfo-icon").span.text
        table["Adress"].append(Address)

        # Using "What people love here" to get food rating, service rating and ambience rating.
        ### Gate 5
        tail_end = soup1.find_all("div" , attrs = {"class" , "col-l-1by3 pl20 pr20"})[1]
        #Additional_Info
        Facility = tail_end.find("div",attrs={"class" : "res-info-highlights"}).text.strip("\n").replace("\n"," ").replace("More Info","").strip()
        table["Facility"].append(Facility)
        # Food rating
        exp = soup1.find_all("div" , class_ = "rv_highlights__score_bar mt5 mb5")
        length = len(exp)
        
        lis1 = exp[0].find_all("div" , class_ = "block level-8")
        n1 = 0
        for element in lis1:
            n1 += 1
        food_rating = n1
        table["Food_Rating"].append(food_rating)

        # Service rating
        lis2 = exp[1].find_all("div" , class_ = "block level-7")
        n2 = 0
        for element in lis2:
            n2 += 1
        service_rating = n2
        table["Service_Rating"].append(service_rating)
        
       
    
        if length == 3:       # Some restaurant don't have data for ambience rating
        
            # Ambience Rating
            lis3 = exp[2].find_all("div" , class_ = "block level-6")
            n3 = 0
            for element in lis3:
                n3+=1
            ambience_rating = n3
            table["Ambience_Rating"].append(ambience_rating)
        else:              # if no ambience rating put NA 
            hue ="NA"
            table["Ambience_Rating"].append(hue)
             #Ask Selenium to click the back button
            #drive.execute_script("window.history.go(-1)") 
        
        
        # Close new tab after getting all data
        driver.close()
        
        
        # Switch handle back to main tab to open next restaurant's link
        driver.switch_to.window(main_window)

In [40]:
table       # take a look at dictionary

{'Name': ['The Cult - Peninsula Grand Hotel',
  'Healthylicious',
  'Rodeo Drive - Peninsula Redpine',
  'One Step Up Bar & Grill',
  'Goa Portuguesa',
  'The Finch',
  'Burma  Burma',
  'Arbab',
  'Lalit Fine Dine',
  'Canto Restaurant & Banquet',
  'London Taxi',
  'HOP : House of Party'],
 'Area': ['Sakinaka',
  'Marol',
  'Marol',
  'Tardeo',
  'Mahim',
  'Powai',
  'Fort',
  'Linking Road, Bandra West',
  'Goregaon West',
  'Lower Parel',
  'Lower Parel',
  'Andheri Lokhandwala, Andheri West'],
 'Type': ['Bar, Lounge ',
  'Quick Bites ',
  'Bar, Casual Dining ',
  'Casual Dining, Bar ',
  'Casual Dining, Bar ',
  'Microbrewery, Casual Dining ',
  'Casual Dining ',
  'Casual Dining ',
  'Casual Dining ',
  'Casual Dining, Bar ',
  'Casual Dining, Bar ',
  'Bar, Casual Dining '],
 'Rating': ['4.5/5',
  '4.5/5',
  '4.5/5',
  '4.5/5',
  '4.5/5',
  '4.6/5',
  '4.5/5',
  '4.6/5',
  '4.5/5',
  '4.5/5',
  '4.7/5',
  '4.5/5'],
 'Ph_Number': ['022 61344389',
  '022 50645844',
  '022 6187686

In [41]:
dataset = pd.DataFrame(table)
dataset.head()

,Name,Area,Type,Rating,Ph_Number,Cuisine,Avg_price,Beer,Time,Adress,Facility,Food_Rating,Service_Rating,Ambience_Rating
0,The Cult - Peninsula Grand Hotel,Sakinaka,"Bar, Lounge",4.5/5,022 61344389,"Continental, North Indian, Chinese","₹2,100",₹345,4:30pm – 1:30am,"Hotel Peninsula Grand, Rooftop, Opposite Sakin...",Wheelchair Accessible Full Bar Available Va...,5,4,0
1,Healthylicious,Marol,Quick Bites,4.5/5,022 50645844,"Healthy Food, Wraps, Burger, Coffee, Juices, S...",₹500,"Cash,\n\n\n\nCards\n",9am – 9pm,"Town Center 1 & 2, Andheri Kurla Road, Opposit...",Home Delivery Vegetarian Only Outdoor Seati...,0,0,NA
2,Rodeo Drive - Peninsula Redpine,Marol,"Bar, Casual Dining",4.5/5,022 61876860,"American, Mexican, Italian, Asian","₹2,000",₹230,6pm – 1:30am,"Peninsula Redpine, Andheri East, Near Airport ...",Full Bar Available Smoking AreaFree ParkingLi...,5,0,0
3,One Step Up Bar & Grill,Tardeo,"Casual Dining, Bar",4.5/5,022 68493378,"North Indian, Continental, Chinese, Italian","₹1,500",₹150,12noon – 1:30am,"1A-C Wing, Raheja Excelsior, Near SBI Bank, Ta...",Full Bar Available Live Sports ScreeningTable...,5,4,4
4,Goa Portuguesa,Mahim,"Casual Dining, Bar",4.5/5,022 61344913,"Seafood, South Indian, Maharashtrian, Goan, No...","₹2,100",₹225,"12noon – 3:30pm, 7pm – 12:30am","Near Hinduja Hospital, Mahim, Mumbai",Home Delivery Wheelchair Accessible Full B...,5,4,0
